In [11]:
import torch  # PyTorch深度学习框架
from scipy.io import loadmat  # 用于加载MATLAB .mat文件
from pathlib import Path  # 路径处理库
import numpy as np  # 科学计算库
from torch.utils.data import DataLoader, TensorDataset  # 数据加载和数据集创建工具
from sklearn.model_selection import train_test_split  # 数据集划分工具
import matplotlib.pyplot as plt  # 可视化库
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix  # 评估指标计算工具
import seaborn as sns  # 高级可视化库
import os  # 操作系统接口
import torch.nn as nn  # PyTorch神经网络模块
import torch.nn.functional as F  # PyTorch函数模块
from torch.optim.lr_scheduler import ReduceLROnPlateau  # 导入学习率调度器

# ============== 数据准备阶段 ==============

# 定义数据路径和模型保存路径
base_path = Path('/home/mw/input/re14334')  # 数据集路径
save_path = Path('checkpoints')  # 模型保存路径
save_path.mkdir(exist_ok=True)  # 创建保存目录，如果已存在则不报错

# 加载 .mat 文件
# river_before: 变化前的河流数据，形状为(463, 241, 198)，表示高度、宽度和光谱波段数
before_data = loadmat(base_path / 'river_before.mat')['river_before']  
# river_after: 变化后的河流数据，形状与before_data相同
after_data = loadmat(base_path / 'river_after.mat')['river_after']     
# groundtruth: 变化标注数据，形状为(463, 241)，二维掩码，255表示变化区域
gt_data = loadmat(base_path / 'groundtruth.mat')['lakelabel_v1']      

# 数据类型转换：将数据转换为PyTorch张量，并设定为float32类型以便计算
before_data = torch.tensor(before_data, dtype=torch.float32)
after_data = torch.tensor(after_data, dtype=torch.float32)
gt_data = torch.tensor(gt_data, dtype=torch.float32)

# 检查数据范围并打印
print(f"Before Data Range: {before_data.min().item()} to {before_data.max().item()}")
print(f"After Data Range: {after_data.min().item()} to {after_data.max().item()}")
print(f"Ground Truth Unique Values: {torch.unique(gt_data)}")

# 改进的归一化方法：按通道归一化
def normalize_per_channel(data):
    """按通道执行归一化，更适合多光谱/高光谱数据"""
    result = data.clone()
    # 对最后一个维度（通道）进行归一化
    for c in range(data.shape[2]):
        channel = data[:, :, c]
        channel_min = channel.min()
        channel_max = channel.max()
        if channel_max > channel_min:  # 避免除以零
            result[:, :, c] = (channel - channel_min) / (channel_max - channel_min)
    return result

before_data = normalize_per_channel(before_data)
after_data = normalize_per_channel(after_data)

# 将标注数据转换为二值掩码：255 -> 1, 其他 -> 0
gt_data = (gt_data == 255).float()

# 调整数据维度顺序以符合PyTorch的约定(通道,高度,宽度)
# 将原始数据(H,W,C)转换为(C,H,W)
before_data = before_data.permute(2, 0, 1)  # 形状变为: (198, 463, 241)
after_data = after_data.permute(2, 0, 1)    # 形状变为: (198, 463, 241)
gt_data = gt_data.unsqueeze(0)              # 添加通道维度，形状变为: (1, 463, 241)

# 特征选择：选择更具辨别力的波段（可以使用专家知识或自动选择）
# 这里简单假设对河流变化最敏感的是前100个波段
selected_bands = list(range(100))  # 假设前100个波段最相关
before_data = before_data[selected_bands]
after_data = after_data[selected_bands]

print(f"数据形状 - Before: {before_data.shape}, After: {after_data.shape}, GT: {gt_data.shape}")

# 改进的patch提取：使用重叠的patch，步长小于patch大小
def extract_patches_with_overlap(image, patch_size=64, stride=32):
    """
    使用重叠的方式提取patches，提高边界区域的预测准确度
    
    参数:
        image: 输入图像张量，形状为(C,H,W)
        patch_size: 每个patch的大小
        stride: 相邻patches的步长
    """
    patches = []
    positions = []  # 记录每个patch的位置
    
    for i in range(0, image.shape[1] - patch_size + 1, stride):
        for j in range(0, image.shape[2] - patch_size + 1, stride):
            patch = image[:, i:i+patch_size, j:j+patch_size]
            patches.append(patch)
            positions.append((i, j))
            
    return torch.stack(patches), positions

# 使用重叠的patch提取
patch_size = 64
stride = 32  # 步长为patch大小的一半，产生重叠
before_patches, patch_positions = extract_patches_with_overlap(before_data, patch_size, stride)
after_patches, _ = extract_patches_with_overlap(after_data, patch_size, stride)
gt_patches, _ = extract_patches_with_overlap(gt_data, patch_size, stride)

print(f"提取的patch数量: {len(before_patches)}")
print(f"Patch形状 - Before: {before_patches.shape}, GT: {gt_patches.shape}")

# 创建数据集并划分
dataset = TensorDataset(before_patches, after_patches, gt_patches)
train_dataset, temp_dataset = train_test_split(dataset, test_size=0.3, random_state=42)
val_dataset, test_dataset = train_test_split(temp_dataset, test_size=0.5, random_state=42)

# 使用较小的batch_size，避免内存问题
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# ============== 改进的模型定义 ==============

class ConvBlock(nn.Module):
    """基本卷积块：Conv2d + BatchNorm + ReLU"""
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=1):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels)
        
    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))

class EncoderBlock(nn.Module):
    """编码器块：两个卷积层 + 最大池化"""
    def __init__(self, in_channels, out_channels):
        super(EncoderBlock, self).__init__()
        self.conv1 = ConvBlock(in_channels, out_channels)
        self.conv2 = ConvBlock(out_channels, out_channels)
        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        pool = self.pool(x)
        return pool, x  # 返回池化后的结果和特征，后者用于跳跃连接

class DecoderBlock(nn.Module):
    """解码器块：上采样 + 两个卷积层"""
    def __init__(self, in_channels, out_channels):
        super(DecoderBlock, self).__init__()
        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.conv1 = ConvBlock(in_channels, out_channels)  # 输入通道数为in_channels，因为要与跳跃连接拼接
        self.conv2 = ConvBlock(out_channels, out_channels)
        
    def forward(self, x, skip):
        x = self.up(x)
        
        # 确保尺寸匹配
        diff_h = skip.size()[2] - x.size()[2]
        diff_w = skip.size()[3] - x.size()[3]
        
        x = F.pad(x, [diff_w // 2, diff_w - diff_w // 2, diff_h // 2, diff_h - diff_h // 2])
        
        # 拼接跳跃连接的特征
        x = torch.cat([skip, x], dim=1)
        
        x = self.conv1(x)
        x = self.conv2(x)
        return x

class ImprovedChangeDetectionModel(nn.Module):
    """改进的变化检测模型：使用UNet架构，加入跳跃连接和注意力机制"""
    def __init__(self, in_channels=100, out_channels=1):
        super(ImprovedChangeDetectionModel, self).__init__()
        
        # 共享编码器
        self.encoder1 = EncoderBlock(in_channels, 64)
        self.encoder2 = EncoderBlock(64, 128)
        self.encoder3 = EncoderBlock(128, 256)
        
        # 差异特征编码器
        self.diff_encoder1 = EncoderBlock(in_channels, 64)
        self.diff_encoder2 = EncoderBlock(64, 128)
        self.diff_encoder3 = EncoderBlock(128, 256)
        
        # 瓶颈层
        self.bottleneck = ConvBlock(256, 512)
        
        # 解码器层
        self.decoder3 = DecoderBlock(512, 256)
        self.decoder2 = DecoderBlock(256, 128)
        self.decoder1 = DecoderBlock(128, 64)
        
        # 输出层
        self.output_conv = nn.Conv2d(64, out_channels, kernel_size=1)
        
    def forward(self, x1, x2):
        # 计算差值图像
        x_diff = torch.abs(x1 - x2)
        
        # 处理第一时相图像
        pool1_1, skip1_1 = self.encoder1(x1)
        pool1_2, skip1_2 = self.encoder2(pool1_1)
        pool1_3, skip1_3 = self.encoder3(pool1_2)
        
        # 处理差值图像
        pool_d1, skip_d1 = self.diff_encoder1(x_diff)
        pool_d2, skip_d2 = self.diff_encoder2(pool_d1)
        pool_d3, skip_d3 = self.diff_encoder3(pool_d2)
        
        # 融合特征
        bottle = self.bottleneck(pool1_3 + pool_d3)
        
        # 解码过程，使用跳跃连接
        up3 = self.decoder3(bottle, skip1_3 + skip_d3)
        up2 = self.decoder2(up3, skip1_2 + skip_d2)
        up1 = self.decoder1(up2, skip1_1 + skip_d1)
        
        # 输出变化概率图
        output = self.output_conv(up1)
        return torch.sigmoid(output)

# 实例化改进的模型
model = ImprovedChangeDetectionModel(in_channels=len(selected_bands))


# ============== 改进的训练设置 ==============

# 使用加权BCE损失处理类别不平衡问题
def weighted_bce_loss(outputs, targets, weight_pos=2.0):
    """
    加权二元交叉熵损失，给正样本（变化区域）更高的权重
    """
    loss = F.binary_cross_entropy(outputs, targets, reduction='none')
    
    # 计算正样本权重
    weights = torch.ones_like(targets)
    weights[targets > 0.5] = weight_pos
    
    # 应用权重
    weighted_loss = weights * loss
    return weighted_loss.mean()

# 定义损失函数和优化器
criterion = weighted_bce_loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # 添加权重衰减防止过拟合
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

# ============== 训练和验证函数 ==============

def train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=50, patience=10):
    """训练函数，包含早停机制和模型保存"""
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_path = save_path / 'best_model.pth'
    history = {
        'train_loss': [], 'val_loss': [], 
        'train_accuracy': [], 'val_accuracy': [],
        'train_precision': [], 'val_precision': [],
        'train_recall': [], 'val_recall': [],
        'train_f1': [], 'val_f1': [],
        'learning_rate': []
    }
    
    for epoch in range(epochs):
        # 训练阶段
        model.train()
        train_loss = 0.0
        train_preds = []
        train_labels = []
        
        for x1, x2, y in train_loader:
            optimizer.zero_grad()
            outputs = model(x1, x2)
            
            # 使用自定义加权损失
            loss = criterion(outputs, y)
            
            loss.backward()
            
            # 梯度裁剪，防止梯度爆炸
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            train_loss += loss.item()
            
            # 收集训练预测结果
            preds = (outputs > 0.5).float()
            train_preds.extend(preds.cpu().numpy().flatten())
            train_labels.extend(y.cpu().numpy().flatten())
        
        train_loss /= len(train_loader)
        
        # 计算训练指标
        train_accuracy = accuracy_score(train_labels, train_preds)
        train_precision = precision_score(train_labels, train_preds)
        train_recall = recall_score(train_labels, train_preds)
        train_f1 = f1_score(train_labels, train_preds)
        
        # 验证阶段
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for x1, x2, y in val_loader:
                outputs = model(x1, x2)
                loss = criterion(outputs, y)
                val_loss += loss.item()
                
                # 收集验证预测结果
                preds = (outputs > 0.5).float()
                val_preds.extend(preds.cpu().numpy().flatten())
                val_labels.extend(y.cpu().numpy().flatten())
                
        val_loss /= len(val_loader)
        
        # 计算验证指标
        val_accuracy = accuracy_score(val_labels, val_preds)
        val_precision = precision_score(val_labels, val_preds)
        val_recall = recall_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds)
        
        # 更新学习率
        scheduler.step(val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        
        # 记录历史
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_accuracy'].append(train_accuracy)
        history['val_accuracy'].append(val_accuracy)
        history['train_precision'].append(train_precision)
        history['val_precision'].append(val_precision)
        history['train_recall'].append(train_recall)
        history['val_recall'].append(val_recall)
        history['train_f1'].append(train_f1)
        history['val_f1'].append(val_f1)
        history['learning_rate'].append(current_lr)
        
        # 打印训练状态
        print(f'\nEpoch {epoch+1}/{epochs}:')
        print(f'Training - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}')
        print(f'Validation - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}')
        print(f'Learning Rate: {current_lr:.6f}')
        
        # 模型保存与早停
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f'Early stopping triggered after {epoch+1} epochs')
            break
    
    # 加载最佳模型
    model.load_state_dict(torch.load(best_model_path))
    
    # 绘制训练历史
    plt.figure(figsize=(15, 10))
    
    # 损失曲线
    plt.subplot(2, 2, 1)
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # 准确率曲线
    plt.subplot(2, 2, 2)
    plt.plot(history['train_accuracy'], label='Training Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy History')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # 精确率和召回率曲线
    plt.subplot(2, 2, 3)
    plt.plot(history['train_precision'], label='Training Precision')
    plt.plot(history['val_precision'], label='Validation Precision')
    plt.plot(history['train_recall'], label='Training Recall')
    plt.plot(history['val_recall'], label='Validation Recall')
    plt.title('Precision and Recall History')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    
    # 学习率曲线
    plt.subplot(2, 2, 4)
    plt.plot(history['learning_rate'])
    plt.title('Learning Rate')
    plt.xlabel('Epoch')
    plt.ylabel('LR')
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.show()
    
    return model

# 训练模型
print("开始训练改进后的模型...")
model = train(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=50, patience=15)

# ============== 评估函数 ==============

def evaluate(model, test_loader):
    """
    评估函数，计算各种性能指标并可视化混淆矩阵
    
    参数:
        model: 待评估的模型
        test_loader: 测试数据加载器
        
    返回:
        包含各项评估指标的字典
    """
    model.eval()  # 设置模型为评估模式
    all_preds = []  # 存储所有预测结果
    all_labels = []  # 存储所有真实标签
    
    # 收集所有预测和标签
    with torch.no_grad():
        for x1, x2, y in test_loader:
            outputs = model(x1, x2)  # 前向传播
            preds = (outputs > 0.5).float()  # 二值化预测结果，阈值为0.5
            
            # 将张量转换为一维数组并添加到列表
            all_preds.extend(preds.cpu().numpy().flatten())
            all_labels.extend(y.cpu().numpy().flatten())
    
    # 计算各种评估指标
    accuracy = accuracy_score(all_labels, all_preds)  # 准确率
    precision = precision_score(all_labels, all_preds)  # 精确率
    recall = recall_score(all_labels, all_preds)  # 召回率
    f1 = f1_score(all_labels, all_preds)  # F1分数
    
    # 绘制混淆矩阵
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    # 返回包含所有指标的字典
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# 在测试集上评估改进后的模型
print("评估改进后的模型...")
metrics = evaluate(model, test_loader)
print("\n测试指标:")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1 Score: {metrics['f1']:.4f}")

# 改进的全图预测函数，使用重叠patch并平均结果
def predict_full_image_improved(model, before_data, after_data, patch_size=64, stride=32):
    """
    改进的全图预测函数，使用重叠的patch并融合结果
    
    参数:
        model: 训练好的模型
        before_data: 变化前的图像数据，形状为(C,H,W)
        after_data: 变化后的图像数据，形状为(C,H,W)
        patch_size: 每个patch的大小
        stride: 相邻patches的步长
    """
    model.eval()
    device = next(model.parameters()).device  # 获取模型所在设备
    
    # 获取原始图像尺寸
    _, height, width = before_data.shape
    
    # 创建预测图和计数图（用于平均重叠区域）
    pred_full = torch.zeros((height, width), device=device)
    count = torch.zeros((height, width), device=device)
    
    # 使用重叠的patch进行预测
    with torch.no_grad():
        for i in range(0, height - patch_size + 1, stride):
            for j in range(0, width - patch_size + 1, stride):
                # 提取patch
                before_patch = before_data[:, i:i+patch_size, j:j+patch_size].unsqueeze(0)
                after_patch = after_data[:, i:i+patch_size, j:j+patch_size].unsqueeze(0)
                
                # 将patch移到模型所在设备
                if before_patch.device != device:
                    before_patch = before_patch.to(device)
                    after_patch = after_patch.to(device)
                
                # 预测
                pred_patch = model(before_patch, after_patch)
                
                # 将结果展平为2D张量 [64, 64]
                pred_patch_flat = pred_patch.squeeze()
                
                # 累加到完整预测图中
                pred_full[i:i+patch_size, j:j+patch_size] += pred_patch_flat
                count[i:i+patch_size, j:j+patch_size] += 1
    
    # 平均重叠区域
    count = torch.clamp(count, min=1)  # 避免除以零
    pred_full = pred_full / count
    
    # 最后进行二值化
    pred_binary = (pred_full > 0.5).float()
    
    # 添加通道维度以匹配期望的输出格式
    pred_binary = pred_binary.unsqueeze(0)
    
    return pred_binary

# 使用改进的全图预测函数
print("对全图进行变化检测预测...")
pred_full = predict_full_image_improved(model, before_data, after_data)
print("预测完成，正在可视化结果...")

# 可视化全图对比结果
def visualize_full_image_comparison(before_data, after_data, gt_data, pred_full):
    """可视化全图对比结果"""
    # 创建四子图布局
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 显示原始图像（使用第一个通道作为示例）
    axes[0, 0].imshow(before_data[0].cpu().numpy(), cmap='gray')
    axes[0, 0].set_title('Before Image (Channel 0)')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(after_data[0].cpu().numpy(), cmap='gray')
    axes[0, 1].set_title('After Image (Channel 0)')
    axes[0, 1].axis('off')
    
    # 显示真实标签和预测结果
    axes[1, 0].imshow(gt_data.squeeze().cpu().numpy(), cmap='gray')
    axes[1, 0].set_title('Ground Truth')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(pred_full.squeeze().cpu().numpy(), cmap='gray')
    axes[1, 1].set_title('Prediction')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.savefig('improved_full_image_comparison.png', dpi=300)
    plt.show()

# 显示全图对比
visualize_full_image_comparison(before_data, after_data, gt_data, pred_full)

# 计算全图的评估指标
pred_binary = pred_full.cpu().numpy().flatten()
gt_binary = gt_data.cpu().numpy().flatten()

accuracy = accuracy_score(gt_binary, pred_binary)
precision = precision_score(gt_binary, pred_binary)
recall = recall_score(gt_binary, pred_binary)
f1 = f1_score(gt_binary, pred_binary)

print("\n改进后的全图评估指标:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# 示例测试区域的可视化
def visualize_predictions(model, test_dataset, num_samples=4):
    """可视化模型预测结果"""
    model.eval()
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    
    for i in range(num_samples):
        x1, x2, y = test_dataset[i]
        
        with torch.no_grad():
            pred = model(x1.unsqueeze(0), x2.unsqueeze(0))
            pred = (pred > 0.5).float()
        
        axes[i, 0].imshow(x1[0].cpu().numpy(), cmap='gray')
        axes[i, 0].set_title('Before')
        axes[i, 0].axis('off')
        
        axes[i, 1].imshow(y.squeeze().cpu().numpy(), cmap='gray')
        axes[i, 1].set_title('Ground Truth')
        axes[i, 1].axis('off')
        
        axes[i, 2].imshow(pred.squeeze().cpu().numpy(), cmap='gray')
        axes[i, 2].set_title('Prediction')
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig('improved_prediction_samples.png', dpi=300)
    plt.show()

# 显示测试样本的预测结果
visualize_predictions(model, test_dataset)

Before Data Range: -142.0 to 7960.0
After Data Range: -227.0 to 7938.0
Ground Truth Unique Values: tensor([  0., 255.])
数据形状 - Before: torch.Size([100, 463, 241]), After: torch.Size([100, 463, 241]), GT: torch.Size([1, 463, 241])
提取的patch数量: 78
Patch形状 - Before: torch.Size([78, 100, 64, 64]), GT: torch.Size([78, 1, 64, 64])
开始训练改进后的模型...

Epoch 1/50:
Training - Loss: 0.6438, Accuracy: 0.7237, Precision: 0.2643, Recall: 0.9592, F1: 0.4145
Validation - Loss: 0.7356, Accuracy: 0.5848, Precision: 0.2240, Recall: 0.9943, F1: 0.3656
Learning Rate: 0.001000

Epoch 2/50:
Training - Loss: 0.4053, Accuracy: 0.9522, Precision: 0.7114, Recall: 0.8939, F1: 0.7922
Validation - Loss: 0.5417, Accuracy: 0.9418, Precision: 0.7112, Recall: 0.8693, F1: 0.7824
Learning Rate: 0.001000

Epoch 3/50:
Training - Loss: 0.3709, Accuracy: 0.9512, Precision: 0.7052, Recall: 0.8962, F1: 0.7893
Validation - Loss: 0.4684, Accuracy: 0.9042, Precision: 0.5733, Recall: 0.7964, F1: 0.6667
Learning Rate: 0.001000

Epoch 4/

<Figure size 1500x1000 with 4 Axes>

评估改进后的模型...


<Figure size 800x600 with 2 Axes>


测试指标:
Accuracy: 0.9833
Precision: 0.9035
Recall: 0.8803
F1 Score: 0.8918
对全图进行变化检测预测...
预测完成，正在可视化结果...


<Figure size 1500x1200 with 4 Axes>


改进后的全图评估指标:
Accuracy: 0.9861
Precision: 0.9426
Recall: 0.8948
F1 Score: 0.9181


<Figure size 1500x2000 with 12 Axes>